# MLLIB Recommender

The classic recommender tutorial uses the [movielens data set](https://grouplens.org/datasets/movielens/). It is similar to using the iris or MNIST data set for other algorithms.


Looking for more datasets? Check out: https://gist.github.com/entaroadun/1653794

In [1]:
import findspark
findspark.init()

In [2]:
from pyspark.sql import SparkSession

In [3]:
spark = SparkSession.builder.appName('rec').getOrCreate()

With Collaborative filtering we make predictions (filtering) about the interests of a user by collecting preferences or taste information from many users (collaborating). The underlying assumption is that if a user A has the same opinion as a user B on an issue, A is more likely to have B's opinion on a different issue x than to have the opinion on x of a user chosen randomly.

The image below (from Wikipedia) shows an example of collaborative filtering. At first, people rate different items (like videos, images, games). Then, the system makes predictions about a user's rating for an item not rated yet. The new predictions are built upon the existing ratings of other users with similar ratings with the active user. In the image, the system predicts that the user will not like the video.

<img src=https://upload.wikimedia.org/wikipedia/commons/5/52/Collaborative_filtering.gif />

Spark MLlib library for Machine Learning provides a Collaborative Filtering implementation by using Alternating Least Squares. The implementation in MLlib has these parameters:

* numBlocks is the number of blocks used to parallelize computation (set to -1 to auto-configure).
* rank is the number of latent factors in the model.
* iterations is the number of iterations to run.
* lambda specifies the regularization parameter in ALS.
* implicitPrefs specifies whether to use the explicit feedback ALS variant or one adapted for implicit feedback data.
* alpha is a parameter applicable to the implicit feedback variant of ALS that governs the baseline confidence in preference observations.

Let's see this all in action!

In [4]:
from pyspark.ml.evaluation import RegressionEvaluator
from pyspark.ml.recommendation import ALS

In [5]:
data = spark.read.csv('movielens_ratings.csv',inferSchema=True,header=True)

In [6]:
data.head()

Row(movieId=2, rating=3.0, userId=0)

In [7]:
data.describe().show()

+-------+------------------+------------------+------------------+
|summary|           movieId|            rating|            userId|
+-------+------------------+------------------+------------------+
|  count|              1501|              1501|              1501|
|   mean| 49.40572951365756|1.7741505662891406|14.383744170552964|
| stddev|28.937034065088994| 1.187276166124803| 8.591040424293272|
|    min|                 0|               1.0|                 0|
|    max|                99|               5.0|                29|
+-------+------------------+------------------+------------------+



We can do a split to evaluate how well our model performed, but keep in mind that it is very hard to know conclusively how well a recommender system is truly working for some topics. Especially if subjectivity is involved, for example not everyone that loves star wars is going to love star trek, even though a recommendation system may suggest otherwise.

In [8]:
# Smaller dataset so we will use 0.8 / 0.2
(training, test) = data.randomSplit([0.8, 0.2])

In [9]:
# Build the recommendation model using ALS on the training data
als = ALS(maxIter=5, regParam=0.01, userCol="userId", itemCol="movieId", ratingCol="rating")
model = als.fit(training)

Now let's see hwo the model performed!

In [10]:
# Evaluate the model by computing the RMSE on the test data
predictions = model.transform(test)

In [11]:
predictions.show()

+-------+------+------+-----------+
|movieId|rating|userId| prediction|
+-------+------+------+-----------+
|     31|   1.0|    27|  1.3832386|
|     31|   1.0|    19|  0.6689191|
|     31|   3.0|     7|  2.3060484|
|     85|   1.0|    26|  4.3885417|
|     85|   5.0|     8|  1.9567294|
|     85|   1.0|     2|-0.26410767|
|     65|   2.0|    15|  2.3844705|
|     53|   1.0|    12|  2.8837388|
|     53|   3.0|    13|  2.9543054|
|     53|   1.0|    25| -2.0672362|
|     78|   1.0|    28| 0.17726241|
|     78|   1.0|    27| 0.28738317|
|     78|   1.0|    12| 0.80764157|
|     78|   1.0|    22|  1.1536131|
|     78|   1.0|    24| 0.97561836|
|     34|   1.0|    14|-0.07214683|
|     34|   1.0|     0| 0.91613096|
|     28|   1.0|    17|  3.6061978|
|     28|   1.0|    23|  0.8081268|
|     28|   5.0|    18|  1.0951443|
+-------+------+------+-----------+
only showing top 20 rows



In [12]:
evaluator = RegressionEvaluator(metricName="rmse", labelCol="rating",predictionCol="prediction")
rmse = evaluator.evaluate(predictions)
print("Root-mean-square error = " + str(rmse))

Root-mean-square error = 1.8133286515751146


The RMSE described our error in terms of the stars rating column.

So now that we have the model, how would you actually supply a recommendation to a user?

The same way we did with the test data! For example:

In [13]:
single_user = test.filter(test['userId']==11).select(['movieId','userId'])

In [15]:
# User had 10 ratings in the test data set 
# Realistically this should be some sort of hold out set!
single_user.show()

+-------+------+
|movieId|userId|
+-------+------+
|     11|    11|
|     13|    11|
|     16|    11|
|     18|    11|
|     22|    11|
|     32|    11|
|     36|    11|
|     59|    11|
|     70|    11|
|     77|    11|
|     99|    11|
+-------+------+



In [16]:
reccomendations = model.transform(single_user)

In [17]:
reccomendations.orderBy('prediction',ascending=False).show()

+-------+------+-----------+
|movieId|userId| prediction|
+-------+------+-----------+
|     18|    11|   4.071852|
|     32|    11|  3.7821665|
|     22|    11|  3.1972456|
|     36|    11|   2.795097|
|     77|    11|  2.3624132|
|     13|    11|  2.2895865|
|     99|    11|  1.2556795|
|     70|    11| 0.24676377|
|     16|    11|-0.14804795|
|     59|    11|-0.47987807|
|     11|    11|-0.91770226|
+-------+------+-----------+

